# Understanding NeuralProphet Forecast Output Structure

This notebook explores the structure of the validation forecasts CSV to understand how to properly index the forecasts.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv('../outputs/validation/validation_forecasts.csv', parse_dates=['ds', 'forecast_time'])
print(f"Shape: {df.shape}")
print(f"Columns: {len(df.columns)}")

## Column Structure

The CSV has the following columns:
- `ds`: Timestamp in the CSV (NOT the target time - see alignment below)
- `yhat1` to `yhat96`: Point forecasts for steps 1-96 ahead
- `yhat1 16.0%` to `yhat96 16.0%`: Lower bound of 68% prediction interval
- `yhat1 84.0%` to `yhat96 84.0%`: Upper bound of 68% prediction interval  
- `y`: Actual observed value at `ds`
- `res1` to `res96`: Residuals for each horizon
- `forecast_time`: Timestamp when the forecast batch was generated

## CRITICAL: Alignment Formula

**To align `yhatN` with actuals, use:**
```python
target_time = ds - 2 * N * freq
```

For example, `yhat12` at `ds=10:30` with 15-min frequency:
- `target_time = 10:30 - 2*12*15min = 10:30 - 6hr = 04:30`
- Plot `yhat12` at x=04:30 to align with actual at 04:30

In [ ]:
# Categorize columns
yhat_cols = [c for c in df.columns if c.startswith('yhat') and '%' not in c]
lower_cols = [c for c in df.columns if '16.0%' in c]
upper_cols = [c for c in df.columns if '84.0%' in c]
res_cols = [c for c in df.columns if c.startswith('res')]

print(f"Point forecast columns (yhat): {len(yhat_cols)}")
print(f"Lower bound columns (16%): {len(lower_cols)}")
print(f"Upper bound columns (84%): {len(upper_cols)}")
print(f"Residual columns: {len(res_cols)}")
print(f"Other columns: ds, y, forecast_time")

## Understanding the Indexing

**IMPORTANT**: `ds` is the forecast ORIGIN time, NOT the target time!

For each forecast batch with `forecast_time`:
- `yhat1` appears at `ds ≈ forecast_time` (rounded to grid), predicts for `ds + 15min`
- `yhat2` appears at `ds + 15min`, predicts for `ds + 30min`  
- ...
- `yhat96` appears at `ds + 23h45m`, predicts for `ds + 24h`

The `yhatN` columns are spread across consecutive rows, with each appearing at `ds = first_ds + (N-1)*freq`.

In [ ]:
# Unique forecast origins
unique_forecast_times = df['forecast_time'].dropna().unique()
print(f"Number of unique forecast origins: {len(unique_forecast_times)}")
print(f"\nFirst 5 forecast times:")
for ft in unique_forecast_times[:5]:
    print(f"  {ft}")

In [ ]:
# Pick first forecast origin and examine
first_ft = unique_forecast_times[0]
print(f"Examining forecast made at: {first_ft}")
print()

# Get rows with yhat values for this forecast
mask = df['forecast_time'] == first_ft
subset = df[mask].copy()

# Find rows with each yhat
print("yhatN -> ds (target time) -> forecast value -> actual y")
print("-" * 70)
for step in [1, 2, 3, 24, 48, 72, 96]:
    col = f'yhat{step}'
    row = subset[subset[col].notna()]
    if len(row) > 0:
        r = row.iloc[0]
        print(f"yhat{step:2d} -> {r['ds']} -> {r[col]:7.3f} -> {r['y']:7.3f}")

## How to Extract Forecasts for a Specific Horizon

**The key insight**: To compare forecast vs actual, you need to align them at the **target time**.

Since `yhatN` at `ds` predicts for `ds + N*freq`, and `y` at `ds` is the actual at `ds` (not at target), 
we use the `resN` column which already contains `y_target - yhatN`:

```
y_target = yhat + residual
```

In [ ]:
def extract_horizon_forecasts(df, horizon, freq='15min'):
    """Extract all forecasts at a specific horizon with PROPERLY ALIGNED target times.
    
    EMPIRICALLY DETERMINED: target_time = ds - 2 * horizon * freq
    
    Args:
        df: DataFrame with validation forecasts
        horizon: Forecast horizon (1-96)
        freq: Frequency string (default '15min')
        
    Returns:
        DataFrame with:
        - ds: Original ds from CSV
        - target_time: Aligned target time = ds - 2*horizon*freq
        - y: Actual value (at ds)
        - yhat: Forecast value
        - lower, upper: Prediction intervals
    """
    freq_delta = pd.Timedelta(freq)
    yhat_col = f'yhat{horizon}'
    lower_col = f'yhat{horizon} 16.0%'
    upper_col = f'yhat{horizon} 84.0%'
    res_col = f'res{horizon}'
    
    # Filter to rows with this horizon's forecast
    mask = df[yhat_col].notna()
    result = df.loc[mask, ['ds', 'y', 'forecast_time']].copy()
    
    # Calculate aligned target time (empirically determined formula)
    result['target_time'] = result['ds'] - 2 * horizon * freq_delta
    
    # Get forecast values
    result['yhat'] = df.loc[mask, yhat_col].values
    result['lower'] = df.loc[mask, lower_col].values if lower_col in df.columns else np.nan
    result['upper'] = df.loc[mask, upper_col].values if upper_col in df.columns else np.nan
    result['residual'] = df.loc[mask, res_col].values if res_col in df.columns else np.nan
    result['horizon'] = horizon
    
    return result.reset_index(drop=True)

# Example: Extract 12-step (3hr) ahead forecasts
h12 = extract_horizon_forecasts(df, 12)
print(f"3-hour ahead forecasts: {len(h12)} samples")
print("\nAlignment check:")
print(f"ds: {h12.iloc[0]['ds']}")
print(f"target_time (ds - 2*12*15min = ds - 6hr): {h12.iloc[0]['target_time']}")
h12[['ds', 'target_time', 'yhat', 'y']].head()

In [ ]:
# Extract forecasts for multiple horizons with proper alignment
horizons = [1, 4, 12, 24, 48, 96]  # 15min, 1hr, 3hr, 6hr, 12hr, 24hr
horizon_labels = ['15min', '1hr', '3hr', '6hr', '12hr', '24hr']

horizon_data = {}
for h, label in zip(horizons, horizon_labels):
    horizon_data[label] = extract_horizon_forecasts(df, h)
    print(f"{label} (step {h}): {len(horizon_data[label])} forecasts")

## Compute Error Metrics by Horizon

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def compute_metrics(data):
    """Compute forecast accuracy metrics.
    
    Note: Uses residual column which is already properly computed in base.py
    """
    valid = data.dropna(subset=['residual', 'yhat'])
    
    if len(valid) == 0:
        return {'MAE': np.nan, 'RMSE': np.nan, 'n_samples': 0}
    
    # MAE from residuals (residual = y_target - yhat, so |residual| = |error|)
    mae = valid['residual'].abs().mean()
    rmse = np.sqrt((valid['residual'] ** 2).mean())
    
    return {'MAE': mae, 'RMSE': rmse, 'n_samples': len(valid)}

# Compute metrics for each horizon
metrics_by_horizon = []
for label, data in horizon_data.items():
    m = compute_metrics(data)
    m['horizon'] = label
    metrics_by_horizon.append(m)

metrics_df = pd.DataFrame(metrics_by_horizon).set_index('horizon')
print("\nMetrics by Forecast Horizon:")
metrics_df

## Visualize Forecasts

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for ax, (label, data) in zip(axes, horizon_data.items()):
    # Compute y_target from residual: y_target = yhat + residual
    valid = data.dropna(subset=['residual', 'yhat']).copy()
    valid['y_target'] = valid['yhat'] + valid['residual']
    
    ax.scatter(valid['y_target'], valid['yhat'], alpha=0.5, s=20)
    
    # Add 1:1 line
    lims = [min(valid['y_target'].min(), valid['yhat'].min()),
            max(valid['y_target'].max(), valid['yhat'].max())]
    ax.plot(lims, lims, 'r--', lw=1)
    
    ax.set_xlabel('Actual')
    ax.set_ylabel('Predicted')
    ax.set_title(f'{label} ahead')
    
    # Add R² annotation
    r2 = np.corrcoef(valid['y_target'], valid['yhat'])[0, 1]**2
    ax.annotate(f'R²={r2:.3f}', xy=(0.05, 0.95), xycoords='axes fraction',
                fontsize=10, verticalalignment='top')

plt.tight_layout()
plt.savefig('figures/forecast_scatter_by_horizon.png', dpi=150)
plt.show()

In [ ]:
# Visualize multiple horizons aligned on the same plot
fig, ax = plt.subplots(figsize=(14, 5))

# Plot actuals (y at ds)
y_data = df[['ds', 'y']].dropna().sort_values('ds')
ax.plot(y_data['ds'], y_data['y'], 'k-', label='Actual', lw=1, alpha=0.7)

# Plot forecasts at different horizons, aligned using: target_time = ds - 2*i*freq
for i in np.arange(4, 32, 8):  # 4, 12, 20, 28 steps = 1hr, 3hr, 5hr, 7hr
    subset = df[['ds', 'y', f'yhat{i}']].dropna()
    target_time = subset['ds'] - 2 * i * pd.Timedelta("15min")
    ax.plot(target_time, subset[f'yhat{i}'], label=f'yhat{i} ({i*15/60:.0f}hr ahead)', alpha=0.7)

ax.set_xlabel('Target Time')
ax.set_ylabel('Temperature (°C)')
ax.legend(loc='upper left')
ax.set_title('Forecasts at Different Horizons (Aligned)')
plt.tight_layout()
plt.savefig('figures/forecast_horizons_aligned.png', dpi=150)
plt.show()

## Alternative: Reshape to Long Format

For easier analysis, we can reshape the wide format to long format:

In [ ]:
def reshape_to_long(df, max_horizon=96, freq='15min'):
    """Reshape the wide forecast data to long format.
    
    Returns DataFrame with columns: ds, target_time, forecast_time, horizon, y, yhat, residual
    """
    freq_delta = pd.Timedelta(freq)
    records = []
    
    for h in range(1, max_horizon + 1):
        yhat_col = f'yhat{h}'
        lower_col = f'yhat{h} 16.0%'
        upper_col = f'yhat{h} 84.0%'
        res_col = f'res{h}'
        
        mask = df[yhat_col].notna()
        if mask.sum() == 0:
            continue
            
        subset = df.loc[mask, ['ds', 'y', 'forecast_time']].copy()
        subset['target_time'] = subset['ds'] - 2 * h * freq_delta
        subset['horizon'] = h
        subset['yhat'] = df.loc[mask, yhat_col].values
        subset['lower'] = df.loc[mask, lower_col].values if lower_col in df.columns else np.nan
        subset['upper'] = df.loc[mask, upper_col].values if upper_col in df.columns else np.nan
        subset['residual'] = df.loc[mask, res_col].values if res_col in df.columns else np.nan
        
        records.append(subset)
    
    return pd.concat(records, ignore_index=True)

# Reshape
long_df = reshape_to_long(df)
print(f"Long format shape: {long_df.shape}")
long_df.head(10)

In [ ]:
# Compute MAE by horizon using residuals (already properly aligned)
mae_by_horizon = long_df.dropna(subset=['residual']).groupby('horizon').apply(
    lambda x: x['residual'].abs().mean()
)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(mae_by_horizon.index, mae_by_horizon.values, 'b-o', markersize=4)
ax.set_xlabel('Forecast Horizon (steps, 15-min each)')
ax.set_ylabel('Mean Absolute Error (°C)')
ax.set_title('Forecast Error vs Horizon')
ax.grid(True, alpha=0.3)

# Add hour markers
for h in [4, 8, 12, 16, 20, 24]:
    step = h * 4
    if step <= 96:
        ax.axvline(step, color='gray', linestyle='--', alpha=0.3)
        ax.text(step, ax.get_ylim()[1], f'{h}h', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig('figures/mae_vs_horizon.png', dpi=150)
plt.show()

## Summary

**The Alignment Formula (Empirically Determined):**

```python
target_time = ds - 2 * horizon * freq
```

For `yhat12` (3hr horizon with 15-min freq):
```
target_time = ds - 2 * 12 * 15min = ds - 6 hours
```

**To plot forecasts aligned with actuals:**
```python
for i in [4, 12, 20, 28]:  # Different horizons
    subset = df[['ds', 'y', f'yhat{i}']].dropna()
    target_time = subset['ds'] - 2 * i * pd.Timedelta("15min")
    plt.plot(target_time, subset[f'yhat{i}'], label=f'yhat{i}')

# Actuals
plt.plot(df['ds'], df['y'], 'k-', label='Actual')
```

**Why the factor of 2?**

This appears to be related to how NeuralProphet indexes multi-step forecasts. The `ds` in the output 
represents a position that is offset from the actual target time by `2 * horizon * freq`.